In [125]:
import re
from collections import Counter, defaultdict
from typing import List, Tuple
import os
import random
import numpy as np
import re


In [140]:
class ngram:
    def __init__(self, n : int, korpus : str, smoothing = False):
        self.n                          = n
        self.smoothing                  = smoothing
        self.toks                       = self._init_toks(korpus)

        self.vocab, self.vocab_count    = self._init_vocab()
        self.vocab_index                = {w:i for i,w in enumerate(self.vocab)}

        self.counts                     = self._count()


    def _init_toks(self, korpus):
        # Alle Interpunktionszeichen außer \w \s ; . ? ! entfernen
        text_no_p = re.sub(r"[^\w\s;.?!]", "", korpus)
        # Leerzeichen vor jedem übrigen Punktionszeichen setzen
        text_no_p = re.sub(r"([;.?!])", r" \1", text_no_p)

        # Satzgrenzen
        text_no_p_ends = re.sub(r"([?!;.])", r"\1 </s>|||<s> ", text_no_p)
        text_no_p_ends = "<s> " + text_no_p_ends + " </s>"

        text_no_p_ends = text_no_p_ends.lower()

        text_no_p_ends_list = re.findall(r'\w+|<s>|</s>|[;.!?]', text_no_p_ends)

        # Each sentence gets a whole list
        toks = []
        group = []

        for tok in text_no_p_ends_list:
            group.append(tok)

            if tok == "</s>":
                toks.append(group)
                group = []

        return toks

    def _init_vocab(self):
        # 1) Zähle alle Tokens
        counter = Counter(tok for sentence in self.toks for tok in sentence)
        
        # 2) Sortiere nach Häufigkeit (absteigend)
        items = counter.most_common()    # Liste von (token, count)
        
        # 3) Entpacke in zwei Listen
        vocab, counts = zip(*items)
        
        return list(vocab), list(counts)


    def _count(self):
        """
        counts: Dict[ Tuple(context), Counter(next_word → count) ]
        """
        counts = defaultdict(Counter)
        for sent in self.toks:
            for i in range(len(sent) - self.n + 1):
                ctx  = tuple(sent[i:i + (self.n - 1)])
                nxt  = sent[i + self.n - 1]
                counts[ctx][nxt] += 1

        if self.smoothing:
            # Add-one Smoothing: für jeden Kontext und jedes Vokabel +1
            for ctx in counts:
                for w in self.vocab:
                    counts[ctx][w] += 1

        return counts
    

    def next_word(self, seed):
        """
        Gibt genau ein Wort zurück, basierend auf dem letzten n-1 Kontext-Tokens.
        """
        toks = seed.lower().split()
        # Links mit <s> füllen, falls Seed zu kurz
        while len(toks) < self.n - 1:
            toks.insert(0, "<s>")
        
        ctx = tuple(toks[-(self.n - 1):])

        # Hol den Counter für diesen Kontext (oder leeren Counter als Fallback)
        counter = self.counts.get(ctx, None)
        if not counter:
            # unbekannter Kontext → gleichverteilt über ganzes Vokabular
            return random.choice(self.vocab)

        # Liste von möglichen Nachfolgern und ihren Counts
        words, weights = zip(*counter.items())

        # Ziehe gewichtet nach Counts (int-weights reichen)
        return random.choices(words, weights=weights, k=1)[0]

    def generate(self, seed: str, length: int) -> str:
        toks = seed.lower().split()
        # Padding links mit <s>
        while self.n > 1 and len(toks) < self.n - 1:
            toks.insert(0, "<s>")

        for _ in range(length):
            nxt = self.next_word(" ".join(toks[-(self.n - 1):]))
            toks.append(nxt)
            if nxt == "</s>":
                break

        # Gib den Text ohne die Start-Markierungen zurück
        return " ".join(toks[self.n - 1:])


In [141]:
# # 10 Random <UNK> for Toks with 1 occourance

# for i, (num, word) in enumerate(num_vocab):
#     if num == 1:
#         begin = i
#         break

# start_end_indexes = list(range(begin, len(num_vocab)))

# choosen = []

# for _ in range(10):
#     rand_idx = random.choice(start_end_indexes)
#     choosen.append(rand_idx)
#     start_end_indexes.remove(rand_idx)

# for i in choosen:
#     num_vocab[i][1] = "<UNK>"

In [142]:
raw_text = ""

for txt in os.listdir("korpus"):
    with open(f"korpus/{txt}", "r", encoding="utf-8") as f:
        content = f.read()

    raw_text += content + " "


In [143]:
LM1 = ngram(1, korpus=raw_text, smoothing=False)

In [144]:
LM2A = ngram(2, korpus=raw_text, smoothing=False)
LM2B = ngram(2, korpus=raw_text, smoothing=True)

In [145]:
LM3A = ngram(3, korpus=raw_text, smoothing=False)
LM3B = ngram(3, korpus=raw_text, smoothing=True)

In [237]:
LM1.next_word("<s> es war")

'ihr'

In [244]:
LM2A.next_word("<s> es war")

';'

In [250]:
LM2B.next_word("<s> es war")

'lobes'

In [178]:
LM2A.generate("<s> es war", 10)

'es war mit freundlicher miene zum lachen und hacken vom sultan von'

In [179]:
LM2B.generate("<s> es war", 10)

'es war asche wort treibst staubwolke wozu wog glücklicherweise unglückliche ausgeruht nebenan'

In [196]:
LM3A.next_word("<s> es war")

'dies'

In [211]:
LM3B.next_word("<s> es war")

'drück'

In [228]:
LM3A.generate("<s> es war", 15)

'war das sichtbar abgenommen hatte . </s>'

In [229]:
LM3B.generate("<s> es war", 15)

'war weniger vorige gelebt verblich bildete bahre rang täglichen berührte idee herausforderung ostern früh eins daher'